In [1]:
# Qpt

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

import numpy.testing as npt

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_backtracking import ProjectedGradientDescentBacktracking, ProjectedGradientDescentBacktrackingOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_measurement,
    get_y_measurement,
    get_z_measurement
)
from quara.objects.gate import (
    Gate, get_depolarizing_channel,
    get_x_rotation,
    get_amplitutde_damping_channel
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

from quara.data_analysis.simulation import StandardQTomographySimulationSetting

from quara.settings import Settings
Settings.get_atol()

1e-13

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
start_all = time.time()

In [4]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# Tester Objects (State)
# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_states = [state_x0, state_y0, state_z0, state_z1]

In [5]:
tester_povms = [get_x_measurement(c_sys), get_y_measurement(c_sys), get_z_measurement(c_sys)]

In [6]:
# True Object
true_objects = []
#true_objects.append(get_depolarizing_channel(p=0, c_sys=c_sys))
true_objects.append(get_depolarizing_channel(p=0.05, c_sys=c_sys))
#true_objects.append(get_depolarizing_channel(p=1, c_sys=c_sys))
#true_objects.append(get_x_rotation(theta=np.pi/2, c_sys=c_sys))
#true_objects.append(get_x_rotation(theta=np.pi, c_sys=c_sys))
#true_objects.append(get_amplitutde_damping_channel(gamma=0.1, c_sys=c_sys))

true_object = true_objects[0]

In [7]:
num_data = [100, 1000]
n_rep = 2

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
    "Maximum-Likelihood(True)",
    "Maximum-Likelihood(False)",
    "Least Squares(True)",
    "Least Squares(False)",
]

seed = 777
qtomography_list = [
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, eps_proj_physical=1e-13, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, eps_proj_physical=1e-13, seed=seed),
]

para_list = [
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropy(12),
    WeightedRelativeEntropy(16),
    WeightedProbabilityBasedSquaredError(12),
    WeightedProbabilityBasedSquaredError(16),
]

loss_option_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropyOption("identity"),
    WeightedRelativeEntropyOption("identity"),
    WeightedProbabilityBasedSquaredErrorOption("identity"),
    WeightedProbabilityBasedSquaredErrorOption("identity"),
]

algo_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
]

algo_option_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktrackingOption(),
    ProjectedGradientDescentBacktrackingOption(),
    ProjectedGradientDescentBacktrackingOption(),
    ProjectedGradientDescentBacktrackingOption(),
]

estimation_results_list = []
elapsed_times = []
simulation_settings = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    loss = loss_list[i]
    loss_option = loss_option_list[i]
    algo = algo_list[i]
    algo_option = algo_option_list[i]

    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       loss=loss,
       loss_option=loss_option,
       algo=algo,
       algo_option=algo_option,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    # stock settings of this simulation
    simulation_setting = StandardQTomographySimulationSetting(
        name=name,
        estimator=estimator,
        loss=loss,
        loss_option=loss_option,
        algo=algo,
        algo_option=algo_option,
        true_object=true_object,
        tester_objects=tester_states + tester_povms
    )
    simulation_settings.append(simulation_setting)

    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

  0%|          | 0/2 [00:00<?, ?it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: LinearEstimator


100%|██████████| 2/2 [00:00<00:00, 11.79it/s]

elapsed_time:0.003308117389678955[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: LinearEstimator


  0%|          | 0/2 [00:00<?, ?it/s]

...

elapsed_time:4.202080619335175[min]

Case 6: Least Squares(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: LossMinimizationEstimator


  0%|          | 0/2 [00:00<?, ?it/s]

elapsed_time:0.6528549631436665[min]

Case 7: Least Squares(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: LossMinimizationEstimator


100%|██████████| 2/2 [00:47<00:00, 23.90s/it]


elapsed_time:0.79692169825236[min]



In [10]:
report.export_report(
    "qpt_nrep=10.pdf",
    estimation_results_list=estimation_results_list,
    simulation_settings=simulation_settings,
    true_object=true_object,
    tester_objects=tester_states+tester_povms,
    seed=seed,
    computation_time=sum(elapsed_times)
)

y.standard.linear_estimator.LinearEstimationResult object at 0x0000024CCC3FC908>], [<quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC3FCB00>, <quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC3FCB38>, <quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC42FE48>, <quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC42F518>, <quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC42FA58>, <quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC42F3C8>, <quara.protocol.qtomography.standard.projected_linear_estimator.ProjectedLinearEstimationResult object at 0x0000024CCC42F390>, <quara.protocol.qtomography

In [10]:
from quara.data_analysis.simulation_check import StandardQTomographySimulationCheck

results = []
for i in range(len(simulation_settings)):
    sim_check = StandardQTomographySimulationCheck(simulation_settings[i], estimation_results_list[i])
    print(f"Case {i}: {simulation_settings[i].name}")
    result = sim_check.execute_all(show_detail=False)
    results.append(result)
    print("-==============================")
    print()

100%|██████████| 2/2 [00:00<00:00, 2532.79it/s]

Case 0: LinearEstimator(True)


TypeError: Unsupported type combination! type=(<class 'quara.objects.gate.Gate'>, <class 'quara.objects.state.State'>)